In [1]:
import json
from pyspark import SparkContext

In [75]:
sc.stop() 

In [4]:

sc = SparkContext(appName="pyspark_si618f22_avg_rating_score_per_category_per_author")

input_file = sc.textFile("/scratch/siads618f22_class_root/siads618f22_class/shared_data/hw6_data/book_ratings_data_jsonl.json")

In [48]:
def get_author_category_and_ratings(data):
    
    author_category_and_ratings_list = []

    # your code here
    rating_scores = float(data["review/score"])
    num_ratings = 1
    num_ratings_active = int(data["User_id"] != None)
    num_ratings_inactive = int(data["User_id"] == None)
    
    if data["authors"] == None:
        authors = ["Unknown"]
    else:
        authors = data["authors"].split(', ')
    
    if data["categories"] == None:
        categories = ["Unknown"]
    else:
        categories = data["categories"].split(', ')

    for author in authors:
        for category in categories:
            if((author!='') &(category!='')):
#                 author = author.replace(' ','')
#                 category = category.replace(' ','')
                cur_tuple = ((author, category),(rating_scores, num_ratings, num_ratings_active, num_ratings_inactive))
                author_category_and_ratings_list.append(cur_tuple)

    return author_category_and_ratings_list

    # hint: author_category_and_ratings_list contains tuples with rating scores, number of ratings, and
    #       number of ratings from active and inactive users for each author-category combination

In [14]:
with open("test.json") as f:
    a = f.readlines()
b = []
for i in a:
    b.append(json.loads(i))
c = get_author_category_and_ratings(b[0])
c

[(('Julie Strain', 'Comics'), (4.0, 1, 1, 0)),
 (('Julie Strain', 'Graphic Novels'), (4.0, 1, 1, 0))]

In [62]:
a = (4.0, 1, 1, 0)
b = (5.0, 2, 3, 1)
c = tuple(map(lambda x,y: x+y, a,b))
d = ("12","23")
'\t'.join(d)

'12\t23'

In [71]:
solution =  input_file.map(lambda line: json.loads(line))\
                      .filter(lambda info: (float(info["review/helpfulness"].split('/')[0]) >= 0.25 * float(info["review/helpfulness"].split('/')[1])) & (info["review/summary"]!= None) & (float(info["review/helpfulness"].split('/')[1])!=0))\
                      .flatMap(lambda info: get_author_category_and_ratings(info))\
                      .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3]))\
                      .map(lambda tup: (tup[0], (round(tup[1][0]/tup[1][1],2), tup[1][1], tup[1][2], tup[1][3])))\
                      .sortBy(lambda tup: (tup[0][0], -tup[1][1], tup[0][1]), ascending = True)\
                      .map(lambda tup: '\t'.join(tup[0])+'\t'+str(tup[1][0])+'\t'+str(tup[1][1])+'\t'+str(tup[1][2])+'\t'+str(tup[1][3]))


In [72]:
solution.take(10)

['A\tJuvenile Fiction\t5.0\t1\t1\t0',
 'A A Acevedo\tGames\t5.0\t1\t1\t0',
 'A A Attanasio\tFiction\t4.74\t42\t31\t11',
 'A A Attanasio\tUnknown\t4.9\t20\t16\t4',
 'A A Cantor\tSuccess in business\t5.0\t1\t1\t0',
 'A A Clarke\tPolitical Science\t4.0\t1\t1\t0',
 'A A F Jochems\tUnknown\t4.0\t2\t1\t1',
 'A A Fair\tPopular literature\t4.0\t1\t1\t0',
 'A A Haidari\tSocial Science\t4.0\t1\t1\t0',
 'A A Harms\tScience\t5.0\t2\t2\t0']

In [73]:
!cat ./sijuntao_si618_hw6_output/part* >  sijuntao_si618_hw6_output.tsv

In [1]:
!diff si618_hw6_preferred_output.tsv sijuntao_si618_hw6_output.tsv